In [ ]:
%load_ext lab_black
import pandas as pd
import numpy as np
import altair as alt
from altair.expr import datum
import helper

In [ ]:
df_1250 = helper.parse_from_file("../sims/1250_sims.csv")
sim1 = helper.SimResults(df_1250)
sim1.df.columns

# Make a interactive plot for all h-cell

In [ ]:
sim1.interactive = (
    sim1.plot_control_space(with_cleanup=True)
    & (
        sim1.plot_mean_dev(show_sd=False, interactive=True, baseline=None)
        | sim1._interactive_wnw(baseline=None)
    )
).properties(title=f"Sim 1 interactive plot (n = {len(sim1.df.code_name.unique())})")

sim1.interactive.save("sim1_interactive.html")

# Make the figures that will be use in draft results

### Figure 1: EoT ACC

In [ ]:
def draw_figure1(df, font_size=18):
    p = (
        alt.Chart(df)
        .mark_rect()
        .encode(
            x=alt.X("p_noise:O", title="P-noise"),
            y=alt.Y("hidden_units:O", sort="descending", title="Hidden"),
            column=alt.Column("learning_rate:O", sort="descending", title="Epsilon"),
            row=alt.Row("cleanup_units:O", sort="descending", title="Cleanup"),
            color=alt.Color(
                "score",
                scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1)),
                title="Accuracy",
            ),
            tooltip=["score"],
        )
        .configure_axis(labelFontSize=font_size, titleFontSize=font_size)
        .configure_header(labelFontSize=font_size, titleFontSize=font_size)
        .configure_legend(labelFontSize=font_size, titleFontSize=font_size)
    )

    return p

#### figure 1a

In [ ]:
df_eot_word = (
    sim1.df.loc[(sim1.df.epoch == 1) & (sim1.df.type == "word"),]
    .groupby(["cell_code", "hidden_units", "cleanup_units", "p_noise", "learning_rate"])
    .mean()
    .reset_index()
)

draw_figure1(df_eot_word).save("part1 plots/fig1a.html")

#### figure 1b

In [ ]:
df_eot_nonword = (
    sim1.df.loc[(sim1.df.epoch == 1) & (sim1.df.type == "nonword"),]
    .groupby(["cell_code", "hidden_units", "cleanup_units", "p_noise", "learning_rate"])
    .mean()
    .reset_index()
)

draw_figure1(df_eot_nonword).save("part1 plots/fig1b.html")

#### Appendix figure 1: each condition

In [ ]:
for c in sim1.df.cond.unique():
    tmp_df = (
        sim1.df.loc[(sim1.df.epoch == 1) & (sim1.df.cond == c),]
        .groupby(
            ["cell_code", "hidden_units", "cleanup_units", "p_noise", "learning_rate",]
        )
        .mean()
        .reset_index()
    )

    draw_figure1(tmp_df).save(f"part1 plots/appendix_fig1_{c}.html")

### Figure 2: Acc over epoch

In [ ]:
def draw_figure2(df, color="cond:N", color_label="Condition", font_size=18):
    p = (
        alt.Chart(df)
        .mark_line()
        .encode(
            x=alt.X("epoch:Q", title="Sample (M)"),
            y=alt.Y("score:Q", title="Accuracy", scale=alt.Scale(domain=(0, 1))),
            color=alt.Color(color, legend=alt.Legend(orient="top"), title=color_label),
        )
        .configure_axis(labelFontSize=font_size, titleFontSize=font_size)
        .configure_header(labelFontSize=font_size, titleFontSize=font_size)
        .configure_legend(labelFontSize=font_size, titleFontSize=font_size)
    )

    return p

#### Figure 2a

In [ ]:
df_dev_wcond = (
    sim1.df.loc[sim1.df.type == "word",].groupby(["cond", "epoch"]).mean().reset_index()
)

draw_figure2(df_dev_wcond).save("part1 plots/fig2a.html")

#### Figure 2b

In [ ]:
df_dev_type = sim1.df.groupby(["type", "epoch"]).mean().reset_index()
draw_figure2(df_dev_type, color="type:N").save("part1 plots/fig2b.html")

#### Figure 2c

In [ ]:
df_dev_hfinc_nwun = (
    sim1.df.loc[sim1.df.cond.isin(["HF_INC", "NW_UN"])]
    .groupby(["cond", "epoch"])
    .mean()
    .reset_index()
)

draw_figure2(df_dev_hfinc_nwun).save("part1 plots/fig2c.html")

### Figure 3: Heatmap word accuracy over epoch

In [ ]:
def draw_figure3(df, font_size=18):
    """ Heatmap h-grid over epoch
    Figure 3 is basically the same as figure 4"""
    p = (
        alt.Chart(df)
        .mark_rect()
        .encode(
            x=alt.X("p_noise:O", title="P-noise"),
            y=alt.Y("hidden_units:O", sort="descending", title="Hidden"),
            column=alt.Column("epoch:O", title="Sample (M)"),
            row=alt.Row("learning_rate:O", sort="descending", title="Epsilon"),
            color=alt.Color(
                "score",
                scale=alt.Scale(scheme="redyellowgreen", domain=(0, 1)),
                title="Accuracy",
            ),
            tooltip=["score"],
        )
        .configure_axis(labelFontSize=font_size, titleFontSize=font_size)
        .configure_header(labelFontSize=font_size, titleFontSize=font_size)
        .configure_legend(labelFontSize=font_size, titleFontSize=font_size)
    )

    return p

In [ ]:
sel_epoch = [0.01, 0.03, 0.05, 0.07, 0.09, 0.2, 0.4, 0.6, 0.8, 1.0]

df_dev_word = (
    sim1.df.loc[(sim1.df.epoch.isin(sel_epoch)) & (sim1.df.type == "word"),]
    .groupby(["cell_code", "hidden_units", "p_noise", "learning_rate", "epoch",])
    .mean()
    .reset_index()
)

draw_figure3(df_dev_word).save("part1 plots/fig3.html")

### Figure 4: Heatmap nonword accuracy over epoch

In [ ]:
sel_epoch = [0.01, 0.03, 0.05, 0.07, 0.09, 0.2, 0.4, 0.6, 0.8, 1.0]

df_dev_nonword = (
    sim1.df.loc[(sim1.df.epoch.isin(sel_epoch)) & (sim1.df.type == "nonword")]
    .groupby(["cell_code", "hidden_units", "p_noise", "learning_rate", "epoch",])
    .mean()
    .reset_index()
)

draw_figure3(df_dev_nonword).save("part1 plots/fig4.html")